In [ ]:
import os
from zipfile import ZipFile
with ZipFile('BONN epilepsy dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/BONN_epilepsy_dataset')
parent_dir = '/content/BONN_epilepsy_dataset'

In [ ]:

parent_dir = '/content/BONN_epilepsy_dataset/BONN epilepsy dataset'
print("Contents of the updated dataset directory:")
print(os.listdir(parent_dir))

Contents of the updated dataset directory:
['F', 'O', 'S']


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import os

parent_dir = '/content/BONN_epilepsy_dataset/BONN epilepsy dataset'
directories = ['S', 'F', 'O']
labels_map = {'S': 'boundary', 'F': 'unknown', 'O': 'safe'}  #label mapping: "safe" and "boundary"

def load_data(parent_dir, directories):
    eeg_data = []
    labels = []

    for dir_name in directories:
        dir_path = os.path.join(parent_dir, dir_name)
        for file_name in sorted(os.listdir(dir_path)):
            file_path = os.path.join(dir_path, file_name)
            with open(file_path, 'r') as file:
                data_points = [int(line.strip()) for line in file]
                eeg_data.append(data_points)
                labels.append(labels_map[dir_name])

    return np.array(eeg_data), np.array(labels)

eeg_data, labels = load_data(parent_dir, directories)

def extract_features(eeg_data):
    """
    Extracts statistical features from EEG data.
    This includes variance, kurtosis, and entropy.
    """
    eeg_data_safe = np.where(eeg_data > 0, eeg_data, 1e-9)

    variance = np.var(eeg_data, axis=1)
    kurtosis = np.mean((eeg_data - np.mean(eeg_data, axis=1, keepdims=True))**4, axis=1)
    entropy = -np.sum(eeg_data_safe * np.log2(eeg_data_safe), axis=1)
    features = np.vstack([variance, kurtosis, entropy]).T
    return features

#kNN boundary detection for F samples
def kNN_boundary_detection(f_data, knn_model):
    """
    Classifies F samples into 'safe', 'boundary', or 'noise' based on kNN model trained on O and S.
    """
    features = extract_features(f_data)
    predictions = knn_model.predict(features)
    return predictions

#training data (only O and S samples)
o_indices = labels == 'safe'
s_indices = labels == 'boundary'

#we combined O and S samples
train_data = np.vstack([eeg_data[o_indices], eeg_data[s_indices]])
train_labels = np.concatenate([labels[o_indices], labels[s_indices]])

train_features = extract_features(train_data)

#trainning the kNN classifier on O and S samples
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_features, train_labels)

#F samples classification
f_indices = labels == 'unknown'
f_data = eeg_data[f_indices]
eeg_types_f = kNN_boundary_detection(f_data, knn)

print("Classifications for F (epileptic non-ictal) samples:", eeg_types_f)


Classifications for F (epileptic non-ictal) samples: ['safe' 'boundary' 'safe' 'safe' 'safe' 'safe' 'safe' 'boundary'
 'boundary' 'boundary' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe'
 'safe' 'safe' 'safe' 'boundary' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe'
 'safe' 'boundary' 'boundary' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe'
 'safe' 'safe' 'safe' 'safe' 'boundary' 'safe' 'safe' 'safe' 'safe' 'safe'
 'safe' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe'
 'safe' 'safe' 'safe' 'boundary' 'boundary' 'safe' 'safe' 'safe' 'safe'
 'safe' 'safe' 'safe' 'safe' 'boundary' 'safe' 'safe' 'boundary' 'safe'
 'safe' 'boundary' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe'
 'safe' 'boundary' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe' 'safe'
 'safe' 'safe' 'safe' 'boundary' 'safe' 'safe']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

latent_dim = 100
data_dim = eeg_data.shape[1]
num_classes = 3  # 0 = safe, 1 = boundary, 2 = noise
batch_size = 64
epochs = 500
lr = 0.0002
beta1 = 0.5

class ConditionalGenerator(nn.Module):
    def __init__(self, latent_dim, data_dim, num_classes):
        super(ConditionalGenerator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        self.model = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, data_dim),
            nn.Tanh()
        )

    def forward(self, z, labels):
        label_embeddings = self.label_embedding(labels)
        input = torch.cat((z, label_embeddings), dim=1)
        return self.model(input)


class ConditionalDiscriminator(nn.Module):
    def __init__(self, data_dim, num_classes):
        super(ConditionalDiscriminator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        self.model = nn.Sequential(
            nn.Linear(data_dim + num_classes, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x, labels):
        label_embeddings = self.label_embedding(labels)
        input = torch.cat((x, label_embeddings), dim=1)
        return self.model(input)


generator = ConditionalGenerator(latent_dim, data_dim, num_classes).to(device)
discriminator = ConditionalDiscriminator(data_dim, num_classes).to(device)
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
criterion = nn.BCELoss()
eeg_data_normalized = (eeg_data - np.min(eeg_data)) / (np.max(eeg_data) - np.min(eeg_data))
eeg_data_normalized = 2 * eeg_data_normalized - 1


def classify_data(eeg_data, labels, k=5):
    from sklearn.neighbors import NearestNeighbors
    neighbors = NearestNeighbors(n_neighbors=k).fit(eeg_data)
    types = []
    distances, indices = neighbors.kneighbors(eeg_data)

    for i, dist in enumerate(indices):
        l = sum(labels[neighbor] == 0 for neighbor in dist)  #we count non-ictal (safe) samples
        if l > 0.8 * k:
            types.append(0)  #safe
        elif 0 < l <= 0.5 * k:
            types.append(1)  #boundary
        else:
            types.append(2)  #noise
    return np.array(types)

real_labels = classify_data(eeg_data_normalized, labels)
eeg_tensor = torch.tensor(eeg_data_normalized, dtype=torch.float32).to(device)
label_tensor = torch.tensor(real_labels, dtype=torch.long).to(device)
dataloader = DataLoader(TensorDataset(eeg_tensor, label_tensor), batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    for real_samples, real_class_labels in dataloader:
        real_samples = real_samples.to(device)
        real_class_labels = real_class_labels.to(device)
        batch_size_curr = real_samples.size(0)
        z = torch.randn(batch_size_curr, latent_dim).to(device)
        fake_class_labels = torch.randint(0, num_classes, (batch_size_curr,), dtype=torch.long).to(device)
        fake_samples = generator(z, fake_class_labels).detach()
        real_targets = torch.ones(batch_size_curr, 1).to(device)
        fake_targets = torch.zeros(batch_size_curr, 1).to(device)
        optimizer_D.zero_grad()
        real_loss = criterion(discriminator(real_samples, real_class_labels), real_targets)
        fake_loss = criterion(discriminator(fake_samples, fake_class_labels), fake_targets)
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()
        z = torch.randn(batch_size_curr, latent_dim).to(device)
        target_class_labels = torch.randint(0, num_classes, (batch_size_curr,), dtype=torch.long).to(device)
        fake_samples = generator(z, target_class_labels)
        optimizer_G.zero_grad()
        g_loss = criterion(discriminator(fake_samples, target_class_labels), real_targets)
        g_loss.backward()
        optimizer_G.step()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

for label in range(num_classes):
    z = torch.randn(1000, latent_dim).to(device)
    labels = torch.full((1000,), label, dtype=torch.long).to(device)
    synthetic_data = generator(z, labels).detach().cpu().numpy()
    print(f"Synthetic data for class {label}:")
    print(synthetic_data)


Epoch [50/500] | D Loss: 0.8072 | G Loss: 3.9067
Epoch [100/500] | D Loss: 0.5875 | G Loss: 2.8797
Epoch [150/500] | D Loss: 0.4286 | G Loss: 3.8682
Epoch [200/500] | D Loss: 0.3984 | G Loss: 6.6218
Epoch [250/500] | D Loss: 0.6660 | G Loss: 3.9887
Epoch [300/500] | D Loss: 0.5915 | G Loss: 7.7392
Epoch [350/500] | D Loss: 0.5246 | G Loss: 4.1647
Epoch [400/500] | D Loss: 0.5689 | G Loss: 4.4039
Epoch [450/500] | D Loss: 0.5956 | G Loss: 3.9689
Epoch [500/500] | D Loss: 0.4361 | G Loss: 6.7721
Synthetic data for class 0:
[[-0.15966456 -0.06263401 -0.14382316 ... -0.15718915 -0.14907733
  -0.09947086]
 [-0.17773975 -0.07334375 -0.15298118 ... -0.17437743 -0.1755515
  -0.12316857]
 [-0.14975886 -0.05308864 -0.1451403  ... -0.144986   -0.14598134
  -0.08725686]
 ...
 [-0.13533655 -0.0512364  -0.12283304 ... -0.13369037 -0.11592359
  -0.06177285]
 [-0.14664648 -0.03151742 -0.12042892 ... -0.15803954 -0.15944616
  -0.09026279]
 [-0.10601423 -0.02527048 -0.09523885 ... -0.11634228 -0.0866646

In [ ]:
import os
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

parent_dir = '/content/BONN_epilepsy_dataset/BONN epilepsy dataset'

def load_bonn_data(parent_dir):
    data = []
    labels = []
    for category_dir in os.listdir(parent_dir):
        category_path = os.path.join(parent_dir, category_dir)
        if os.path.isdir(category_path):
            label = len(labels)
            for file in os.listdir(category_path):
                file_path = os.path.join(category_path, file)
                sample = np.loadtxt(file_path)
                data.append(sample)
                labels.append(label)
    return np.array(data), np.array(labels)

real_data, real_labels = load_bonn_data(parent_dir)

X_train, X_test, y_train, y_test = train_test_split(real_data, real_labels, test_size=0.2, random_state=42)
k = 10  #number of neighbors
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

print("Evaluation on Real Data:")
y_pred_real = knn.predict(X_test)
print(classification_report(y_test, y_pred_real))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_real))


Evaluation on Real Data:
              precision    recall  f1-score   support

           0       0.38      1.00      0.55        22
         100       0.00      0.00      0.00        16
         200       1.00      0.09      0.17        22

    accuracy                           0.40        60
   macro avg       0.46      0.36      0.24        60
weighted avg       0.51      0.40      0.26        60

Confusion Matrix:
 [[22  0  0]
 [16  0  0]
 [20  0  2]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import numpy as np
import torch
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def generate_synthetic_data(class_label, num_samples):
    """
    Generate synthetic data using the GAN.
    Args:
        class_label (int): The class for which data is to be generated.
        num_samples (int): Number of samples to generate.
    Returns:
        numpy.ndarray: Synthetic data of shape (num_samples, num_features).
    """
    generator.eval()
    z = torch.randn(num_samples, latent_dim).to(device)
    labels = torch.full((num_samples,), class_label, dtype=torch.long).to(device)
    with torch.no_grad():
        synthetic_data = generator(z, labels).detach().cpu().numpy()

    return synthetic_data

num_synthetic_samples = 200
synthetic_data = []
synthetic_labels = []

#we generate synthetic data for each class
for class_label in [0, 1, 2]:  #classes: 0 = safe 1 = boundary 2 = noise
    data = generate_synthetic_data(class_label, num_synthetic_samples)
    synthetic_data.append(data)
    synthetic_labels.append(np.full(num_synthetic_samples, class_label))
synthetic_data = np.vstack(synthetic_data)
synthetic_labels = np.concatenate(synthetic_labels)

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
y_pred_synthetic = knn.predict(synthetic_data)
print("Evaluation on Synthetic Data:")
print(classification_report(synthetic_labels, y_pred_synthetic))
print("Confusion Matrix:\n", confusion_matrix(synthetic_labels, y_pred_synthetic))


Evaluation on Synthetic Data:
              precision    recall  f1-score   support

           0       0.33      1.00      0.50       200
           1       0.00      0.00      0.00       200
           2       0.00      0.00      0.00       200

    accuracy                           0.33       600
   macro avg       0.11      0.33      0.17       600
weighted avg       0.11      0.33      0.17       600

Confusion Matrix:
 [[200   0   0]
 [200   0   0]
 [200   0   0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
